In [31]:
import pandas as pd
import numpy as np
from plotly import graph_objects as go
import re
from plotly.subplots import make_subplots
# import plotly.express as px

In [32]:
df_salaries = pd.read_csv('C:\\Users\\iKerng\\Python\\School_DA\\final_task\\data\\Salaries.csv', delimiter=',')
df_salaries = df_salaries[['Id', 'EmployeeName', 'JobTitle', 'BasePay', 'OvertimePay',
                           'OtherPay', 'Benefits', 'TotalPay', 'TotalPayBenefits', 'Year', 'Status']]
df_work = df_salaries.copy()

C:\Users\iKerng\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning:

Columns (3,4,5,6,12) have mixed types.Specify dtype option on import or set low_memory=False.



In [33]:
def str_nice(name=''):
    if type(name) == str:
        name = name.lower()
        name = re.sub(r'\s{2,}', ' ', name)
        name = re.sub(r'[^\w ]|\d', '', name)
    return name

In [34]:
df_work.EmployeeName = df_work.EmployeeName.apply(str_nice)
df_work.JobTitle = df_work.JobTitle.apply(str_nice)
df_work = df_work[df_work.EmployeeName != 'not provided']
df_work.BasePay = df_work.BasePay.astype(float)
df_work.OvertimePay = df_work.OvertimePay.astype(float)
df_work.OtherPay = df_work.OtherPay.astype(float)
df_work.Benefits = df_work.Benefits.astype(float)
df_work.TotalPay = df_work.TotalPay.astype(float)
df_work.TotalPayBenefits = df_work.TotalPayBenefits.astype(float)

In [35]:
ls_job_titles = df_work[~(df_work.JobTitle.str.contains('fire')
        | df_work.JobTitle.str.contains('police')
        | df_work.JobTitle.str.contains('medicine'))].JobTitle.apply(lambda x: re.split(' ', x)).to_list()
job_title_words = []
for i in ls_job_titles:
    for k in i:
        job_title_words.append(k)
(pd.DataFrame(job_title_words, columns=['job_words']).
 groupby(by='job_words')[['job_words']].count()).rename(columns={'job_words':'quantity'}).\
    reset_index().job_words.to_list()
#
# ls_clean_words = ['acupuncturist', 'anesthetist', 'captain', 'clincal', 'clinical', 'commissioner', 'criminalist',
#  'dental', 'dentist', 'diagnostic', 'dietetic', 'dietitian', 'disease', 'emerg', 'emergency', 'emergencycy',
#  'epidemiologist', 'forensic', 'hlth', 'hospital', 'hygienist', 'injury', 'inpsector', 'inspection',
#  'inspections', 'inspector', 'fire', 'police', 'medicine']
ls_clean_words = ['acupuncturist', 'anesthetist', 'clincal', 'clinical', 'dental', 'dentist', 'diagnostic', 'dietetic'
    , 'dietitian', 'disease', 'emerg', 'emergency', 'emergencycy', 'epidemiologist', 'forensic', 'hlth', 'hospital'
    , 'hygienist', 'injury', 'sheriff', 'captain', 'criminalist', 'fire', 'police', 'medic']
str_contains_words = str(ls_clean_words).replace("['", '').replace("', '", '|').replace("']", '').replace(' ', '')

In [36]:
ls_clean_words_med = ['acupuncturist', 'anesthetist', 'clincal', 'clinical', 'dental', 'dentist', 'diagnostic'
    , 'dietetic', 'dietitian', 'disease', 'emerg', 'emergency', 'emergencycy', 'epidemiologist', 'forensic', 'hlth'
    , 'hospital', 'hygienist', 'medic']
ls_clean_words_fire = ['fire']
ls_clean_words_police = ['sheriff', 'criminalist', 'police']

str_word_med = str(ls_clean_words_med).replace("['", '').replace("', '", '|').replace("']", '').replace(' ', '')
str_word_fire = str(ls_clean_words_fire).replace("['", '').replace("', '", '|').replace("']", '').replace(' ', '')
str_word_police = str(ls_clean_words_police).replace("['", '').replace("', '", '|').replace("']", '').replace(' ', '')

ls_fire = df_work[df_work.JobTitle.str.contains(str_word_fire)].index.to_list()
ls_med = df_work[df_work.JobTitle.str.contains(str_word_med)
        & ~(df_work.JobTitle.str.contains(str_word_fire)
            | df_work.JobTitle.str.contains(str_word_police))].index.to_list()
ls_police = df_work[df_work.JobTitle.str.contains(str_word_police)
        & ~(df_work.JobTitle.str.contains(str_word_med) | df_work.JobTitle.str.contains(str_word_fire))].index.to_list()

df_work['department'] = None
df_work.department.iloc[ls_fire] = 'fire'
df_work.department.iloc[ls_med] = 'medic'
df_work.department.iloc[ls_police] = 'police'

C:\Users\iKerng\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [37]:
df_work_dep = df_work[df_work.department.notna()].copy()
df_work_dep = df_work_dep[['department', 'Year', 'BasePay']]
df_work_graf = df_work_dep.groupby(by=['department', 'Year']).aggregate([max, 'median', 'mean'])['BasePay'].reset_index()
df_work_graf = df_work_graf.rename(columns={'max':'Maximum', 'median':'Median', 'mean':'Average'})

### 1) Изменение оклада сотрудников различных департаментов (максимальная, медианная и средняя).

In [38]:
df_work_graf.sort_values(by=['department', 'Year'])

,department,Year,Maximum,Median,Average
0,fire,2011,285262.00,105934.670,106950.754469
1,fire,2012,302578.00,121524.030,120133.939144
2,fire,2013,315572.01,128138.210,129291.774625
3,fire,2014,311298.55,118898.250,120959.820794
4,medic,2011,257510.59,59923.625,66433.847404
5,medic,2012,217483.34,62611.710,67781.535768
6,medic,2013,270641.50,66712.190,71730.393759
7,medic,2014,267914.00,62892.510,67531.008628
8,police,2011,261717.60,106015.310,98840.495313
9,police,2012,195242.05,102879.375,93216.607836


In [39]:
for i in ['Maximum','Median','Average']:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_work_graf[df_work_graf.department == 'police']['Year'],
                             y=df_work_graf[df_work_graf.department == 'police'][i], name='Police Department'))
    fig.add_trace(go.Scatter(x=df_work_graf[df_work_graf.department == 'fire']['Year'],
                             y=df_work_graf[df_work_graf.department == 'fire'][i], name='Fire Department'))
    fig.add_trace(go.Scatter(x=df_work_graf[df_work_graf.department == 'medic']['Year'],
                             y=df_work_graf[df_work_graf.department == 'medic'][i], name='Medical Department'))
    fig.update_layout(title=i + ' BasePay by year',
                      xaxis_title='Year',
                      yaxis_title='Basepay $')
    fig.show()

In [40]:
df_infl_rate = pd.DataFrame([[2010, 1.64], [2011, 3.16], [2012, 2.07], [2013, 1.46], [2014, 1.62], [2015, 0.12]]
                            , columns=['Year','inflation'])
df_work_graf = df_work_graf.merge(df_infl_rate, on='Year')

In [41]:
df_work_graf_infl = df_work_graf.copy()

df_work_graf_infl.Year.unique().min()
df_work_graf_infl['maximum_with_inflation'] = float(0)
df_work_graf_infl['median_with_inflation'] = float(0)
df_work_graf_infl['average_with_inflation'] = float(0)
for year in df_work_graf_infl.Year.unique():
    ls_year_index = df_work_graf_infl[df_work_graf_infl.Year == year].index.to_list()
    if year == df_work_graf_infl.Year.unique().min():
        df_work_graf_infl['maximum_with_inflation'].iloc[ls_year_index] = df_work_graf_infl.Maximum.iloc[ls_year_index]
        df_work_graf_infl['median_with_inflation'].iloc[ls_year_index] = df_work_graf_infl.Median.iloc[ls_year_index]
        df_work_graf_infl['average_with_inflation'].iloc[ls_year_index] = df_work_graf_infl.Average.iloc[ls_year_index]
    else:
        current_infl_rate = df_infl_rate[df_infl_rate.Year == year]['inflation'].tolist()[0]
        ls_year_index_prev = df_work_graf_infl[df_work_graf_infl.Year == (year - 1)].index.to_list()
        df_work_graf_infl['maximum_with_inflation'].iloc[ls_year_index] = df_work_graf_infl.maximum_with_inflation.\
            iloc[ls_year_index_prev].apply(lambda x: x*(1 + current_infl_rate/100)).to_list()
        df_work_graf_infl['median_with_inflation'].iloc[ls_year_index] = df_work_graf_infl.median_with_inflation.\
            iloc[ls_year_index_prev].apply(lambda x: x*(1 + current_infl_rate/100)).to_list()
        df_work_graf_infl['average_with_inflation'].iloc[ls_year_index] = df_work_graf_infl.average_with_inflation.\
            iloc[ls_year_index_prev].apply(lambda x: x*(1 + current_infl_rate/100)).to_list()


C:\Users\iKerng\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\iKerng\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\iKerng\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\iKerng\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWi

### 2) График соотнесения фактического роста зарплат сотрудников с ростом зарплат сотрудников засчет индексации по уровню инфляции

In [42]:
df_work_graf_infl.sort_values(by=['department', 'Year'])

,department,Year,Maximum,Median,Average,inflation,maximum_with_inflation,median_with_inflation,average_with_inflation
0,fire,2011,285262.00,105934.670,106950.754469,3.16,285262.000000,105934.670000,106950.754469
3,fire,2012,302578.00,121524.030,120133.939144,2.07,291166.923400,108127.517669,109164.635086
6,fire,2013,315572.01,128138.210,129291.774625,1.46,295417.960482,109706.179427,110758.438759
9,fire,2014,311298.55,118898.250,120959.820794,1.62,300203.731441,111483.419534,112552.725466
1,medic,2011,257510.59,59923.625,66433.847404,3.16,257510.590000,59923.625000,66433.847404
4,medic,2012,217483.34,62611.710,67781.535768,2.07,262841.059213,61164.044037,67809.028046
7,medic,2013,270641.50,66712.190,71730.393759,1.46,266678.538678,62057.039080,68799.039855
10,medic,2014,267914.00,62892.510,67531.008628,1.62,270998.731004,63062.363114,69913.584301
2,police,2011,261717.60,106015.310,98840.495313,3.16,261717.600000,106015.310000,98840.495313
5,police,2012,195242.05,102879.375,93216.607836,2.07,267135.154320,108209.826917,100886.493566


In [43]:
for i in [['Maximum', 'maximum_with_inflation'],
          ['Median', 'median_with_inflation'],
          ['Average', 'average_with_inflation']]:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_work_graf_infl[df_work_graf_infl.department == 'police']['Year'],
                             y=df_work_graf_infl[df_work_graf_infl.department == 'police'][i[0]]
                             , name='Police Department'))
    fig.add_trace(go.Scatter(x=df_work_graf_infl[df_work_graf_infl.department == 'police']['Year'],
                             y=df_work_graf_infl[df_work_graf_infl.department == 'police'][i[1]]
                             , name='Police Department with inflation'))
    fig.add_trace(go.Scatter(x=df_work_graf_infl[df_work_graf_infl.department == 'fire']['Year'],
                             y=df_work_graf_infl[df_work_graf_infl.department == 'fire'][i[0]]
                             , name='Fire Department'))
    fig.add_trace(go.Scatter(x=df_work_graf_infl[df_work_graf_infl.department == 'fire']['Year'],
                             y=df_work_graf_infl[df_work_graf_infl.department == 'fire'][i[1]]
                             , name='Fire Department with inflation'))
    fig.add_trace(go.Scatter(x=df_work_graf_infl[df_work_graf_infl.department == 'medic']['Year'],
                             y=df_work_graf_infl[df_work_graf_infl.department == 'medic'][i[0]]
                             , name='Medical Department'))
    fig.add_trace(go.Scatter(x=df_work_graf_infl[df_work_graf_infl.department == 'medic']['Year'],
                             y=df_work_graf_infl[df_work_graf_infl.department == 'medic'][i[1]]
                             , name='Medical Department with inflation'))
    fig.update_layout(title=i[0] + ' BasePay by year with inflation rate',
                      xaxis_title='Year',
                      yaxis_title='Basepay $')
    fig.show()


### 3) Графики распределения зарплат/переработки/льгот сотрудников в разрезе департаментов

In [44]:
for i in (df_work[['BasePay', 'OvertimePay', 'Benefits']]).columns:
    kolonka = 0
    fig = make_subplots(rows=1, cols=4)
    for year in df_work.Year.unique():
        kolonka += 1
        fig.add_trace(go.Histogram(x=df_work[(df_work.department == 'fire')
                                             & (df_work[i] != 0)
                                             & (df_work['Year'] == year)][i], name='Fire department ' + i + str(year))
                      , row=1, col=kolonka)
        fig.add_trace(go.Histogram(x=df_work[(df_work.department == 'police')
                                             & (df_work[i] != 0)
                                             & (df_work['Year'] == year)][i], name='Police department ' + i + str(year))
                      , row=1, col=kolonka)
        fig.add_trace(go.Histogram(x=df_work[(df_work.department == 'medic')
                                             & (df_work[i] != 0)
                                             & (df_work['Year'] == year)][i], name='Medical department ' + i + str(year))
                      , row=1, col=kolonka)
        fig.update_xaxes(title=i + ' of ' + str(year), col=kolonka, row=1)
        fig.update_xaxes(title=i + ' of ' + str(year), col=kolonka, row=1)
        fig.update_xaxes(title=i + ' of ' + str(year), col=kolonka, row=1)
        fig.update_xaxes(title=i + ' of ' + str(year), col=kolonka, row=1)
    fig.update_layout(title=i + ' density by departments by ' + str(df_work.Year.unique().tolist())[1:-1] + ' years',
                      # xaxis_title=i,
                      yaxis_title='Count Employee',
                      height=600,
                      width=1500,
                      legend_orientation='h')
    fig.show()


In [45]:
for i in (df_work[['BasePay', 'OvertimePay', 'Benefits']]).columns:
    kolonka = 0
    fig = make_subplots(rows=1, cols=3)
    for dep in df_work[df_work.department.notna()].department.unique():
        kolonka += 1
        fig.add_trace(go.Histogram(x=df_work[(df_work.department == dep)
                                             & (df_work[i] != 0)
                                             & (df_work.Year == 2011)][i]
                                   , name= dep + ' ' + i + ' 2011'
                                   )
                      , row=1
                      , col=kolonka)
        fig.add_trace(go.Histogram(x=df_work[(df_work.department == dep)
                                             & (df_work[i] != 0)
                                             & (df_work.Year == 2012)][i]
                                   , name= dep + ' ' + i + ' 2012'
                                   )
                      , row=1
                      , col=kolonka)
        fig.add_trace(go.Histogram(x=df_work[(df_work.department == dep)
                                             & (df_work[i] != 0)
                                             & (df_work.Year == 2013)][i]
                                   , name= dep + ' ' + i + ' 2013'
                                   )
                      , row=1
                      , col=kolonka)
        fig.add_trace(go.Histogram(x=df_work[(df_work.department == dep)
                                             & (df_work[i] != 0)
                                             & (df_work.Year == 2014)][i]
                                   , name= dep + ' ' + i + ' 2014'
                                   # , legendgroup='h'
                                   )
                      , row=1
                      , col=kolonka)
        fig.update_xaxes(title=i + ' of ' + dep + ' department', col=kolonka, row=1)
        fig.update_xaxes(title=i + ' of ' + dep + ' department', col=kolonka, row=1)
        fig.update_xaxes(title=i + ' of ' + dep + ' department', col=kolonka, row=1)
    fig.update_layout(title=i + ' density by Police/Fire/Medical departments by years',
                      yaxis_title='Count Employee',
                      height=600,
                      width=1500,
                      legend_orientation='h')
    fig.show()

### 4) График распределения бюджета по департаментам в разрезе лет

In [46]:
df_four = (df_work[df_work.department.notna()][['department', 'Year', 'TotalPayBenefits']]).\
    groupby(by=['department', 'Year'])[['TotalPayBenefits']].sum().reset_index()
fig = go.Figure()
for year in df_four.Year.unique():
    fig.add_trace(go.Bar(x=df_four[df_four.Year == year].department, y=df_four[df_four.Year == year].TotalPayBenefits,
                         name= str(year)))
    fig.update_layout(title='Total pay with benefits by department by ' + str(year) + ' years',
          xaxis_title='Total Pay Department',
          yaxis_title='Department')
fig.show()

### 5) Какое количество людей в каждом департаменте?

In [47]:
df_dep_count_employee = (df_work[df_work.department.notna()].groupby(by=['department', 'EmployeeName'])[['department']].count().
reset_index('EmployeeName')[['EmployeeName']]).reset_index().groupby(by='department')[['department']].count().\
    rename(columns={'department':'quantity'}).reset_index()
df_dep_count_employee

,department,quantity
0,fire,5184
1,medic,3815
2,police,10564


In [48]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df_dep_count_employee['department'], y=df_dep_count_employee['quantity']))
fig.update_layout(xaxis_title='Department', yaxis_title='Quantity', title='Count employee by department')
fig.show()
df_dep_count_employee

,department,quantity
0,fire,5184
1,medic,3815
2,police,10564


### 6) Количество сотрудников по доходам с шагом в 5000$

In [49]:
min_max_bp = df_work[df_work.BasePay >= 0].BasePay.aggregate(['min', 'max']).tolist()
delta = min_max_bp[1] - min_max_bp[0]
num_bins = np.ceil(delta / 5000)
end_value = int(num_bins*5000+1)
df_six = df_work[df_work.BasePay >= 0][['Year', 'BasePay', 'EmployeeName']].\
    groupby(by=['Year', 'EmployeeName'])[['BasePay']].sum().reset_index()
df_six['category'] = pd.cut(df_six[df_six.BasePay >= 0].BasePay, bins=range(0, end_value, 5000), right=False)

df_six_grouped = df_six[['Year', 'category']].groupby(by=['Year', 'category'])[['category']].count().\
    rename(columns={'category':'quantity'}).reset_index()
df_six_grouped.quantity.iloc[df_six_grouped[df_six_grouped.quantity.isna()].index.to_list()] = 0
df_six_grouped.quantity = df_six_grouped.quantity.astype(int)
df_six_grouped.category = df_six_grouped.category.astype(str)
df_six_grouped

C:\Users\iKerng\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Year,category,quantity
0,2011,"[0, 5000)",2583
1,2011,"[5000, 10000)",1528
2,2011,"[10000, 15000)",1110
3,2011,"[15000, 20000)",799
4,2011,"[20000, 25000)",636
...,...,...,...
251,2014,"[295000, 300000)",0
252,2014,"[300000, 305000)",1
253,2014,"[305000, 310000)",1
254,2014,"[310000, 315000)",1


In [50]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df_six_grouped[df_six_grouped.Year == 2011]['category'],
                     y=df_six_grouped[df_six_grouped.Year == 2011]['quantity']))
fig.update_layout(title='Count employee by category BasePay with step 5000$',
                  xaxis_title='Interval of BasePay',
                  yaxis_title='Count Employee',
                  height=600,
                  width=1500)

fig.show()